## Project Kijkcijfers
Onderzoeken welke facturen de kijkcijfers beïnvloeden alsook kijkcijfers voorspellen.

### Stappenplan
1. Probleemstelling
2. Data gathering
3. Data visualiseren en analyseren
4. Data cleaning
5. Model selection and training
6. Fine tune model
6. Launch & maintain

# Probleemstelling
Onderzoeken welke facturen de kijkcijfers beïnvloeden alsook kijkcijfers voorspellen.

# Data gathering
Data van de kijkcijfers is de vinden op CIM. We maken een klein python scriptje die iteratief de data van via API calls ophaalt bij CIM en vervolgens opslaat in een csv file. zie 'gather-views.py'

1ste probleem: data voor 14 en 15 augustus 2019 ontbreken. We passen het script aan zodat het niet stopt als er 1 dag geen data gevonden wordt, maar pas stopt als het 5 achtereenvolgende dagen geen data meer kan ophalen.

Data wordt opgeslaan in 'tv_views_data.csv'

Sidenote: Ik heb er aan gedacht om de weersvoorspelling toe te voegen aan deze datalijst maar in mijn opinie zorgt dit voor een extra probabiliteit en hebben we meer info door gebruik te maken van seizoenen. Uit mijn ervaring is de weersvoorspelling in België een extra factor van onzekerheid en daarom geen goede maatstaf om toe te voegen aan het model.

# Data visualisatie
Hier gaan we de data uit de csv visualiseren en onderzoeken. Kijken waar we bepaalde zaken kunnen opkuisen en eventueel al enkele patronen kunnen opmerken.

Hieronder importeren we eerst enkele belangrijke packages alsook een check van de python versie zoals we in elke les deden.

In [2]:
# Python ≥3.11 is required
import sys
assert sys.version_info >= (3, 11)

# Scikit-Learn ≥ 1.6 is required
from packaging import version
import sklearn
print(sklearn.__version__)
assert version.parse(sklearn.__version__) >= version.parse("1.6")

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt

1.6.1


We lezen de data en in bekijken de eerste 10 records, daarna bekijken we de info van de algehele tabel.

In [3]:
tv_views = pd.read_csv('tv_views_data.csv')
tv_views.head()


,description,category,channel,startTime,rLength,rateInK,weather,temperature
THUIS,NaN,VRT 1,20:24:14,00:24:08,873.935,NaN,NaN,2025-04-03
HET 7 UUR-JOURNAAL,NaN,VRT 1,19:00:05,00:48:22,859.904,NaN,NaN,2025-04-03
MAN BIJT HOND,NaN,VRT 1,19:51:09,00:24:55,702.872,NaN,NaN,2025-04-03
NIEUWS 19U VTM,NaN,VTM,18:59:50,00:59:10,562.650,NaN,NaN,2025-04-03
BLOKKEN,NaN,VRT 1,18:29:28,00:28:30,478.774,NaN,NaN,2025-04-03


Als we de info van de lijst bekijken zien we dat er heel wat null values zijn. Ik wil eerst alle data bekijken alvorens ik dit ga opkuisen. Zo krijg ik al een eerste visueel beeld van de data alvorens ik vanalles ga verwijderen. Ookal ben ik vrijwel zeker dat de null values weg mogen.

In [4]:
tv_views.info()


<class 'pandas.core.frame.DataFrame'>
Index: 62459 entries, THUIS to MOUNTAINBIKE. WB. FIN TRIAL ANTWERPEN
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   description  0 non-null      float64
 1   category     62373 non-null  object 
 2   channel      62371 non-null  object 
 3   startTime    62370 non-null  object 
 4   rLength      62370 non-null  object 
 5   rateInK      0 non-null      float64
 6   weather      0 non-null      float64
 7   temperature  62459 non-null  object 
dtypes: float64(3), object(5)
memory usage: 4.3+ MB


Eerst doen we een algemene visualisatie van de data. Daarna wil ook wat dieper ingaan op de data rond de Covid-19 lockdown periodes. Volgens mij zouden de kijkcijfers daar iets hoger liggen dan het gemiddelde. Afhankelijk van deze observatie moet ik de beslissing maken of ik deze data als uitschieter benoem of niet. Want laat ons hopen dat zo'n lockdown niet meer terugkomt.

Periode waar horeca gesloten was:
- 13 maart 2020 tot 8 juni 2020
- 16 oktober 2020 tot 11 mei 2021